In [1]:
##====================================================
## このセルを最初に実行せよ---Run this cell initially.
##====================================================
import sys
if 'google.colab' in sys.modules:
    !wget -P ./text https://www.eidos.ic.i.u-tokyo.ac.jp/~sato/assignments/project2/text/test_data.csv
    !wget -P ./text https://www.eidos.ic.i.u-tokyo.ac.jp/~sato/assignments/project2/text/wiki_dataset.csv

# ミニプロジェクト（発展課題） / Miniproject (Advanced exercises)

## Project2. 自分のアイディアで手法を改良しよう（発展課題）

基礎課題では、ウィキペディアの6カテゴリの記事からなるデータセット $D$ を学習データとして、カテゴリが未知の記事6本を分類しました。   
これら6本では正しく分類できたと思いますが、いろいろな記事で試してみると、正しく分類できない記事もあることがわかります。   

そこで発展課題では、基礎課題で実装した手法をベースライン（基準）とし、それよりも高い精度で分類できるよう、手法を改良してください。    
**皆さん自身で考えたアイディアを実装**し、**ベースラインの手法と皆さんの提案手法とで精度を比較評価した結果を報告**してください。  
適宜、図や表を使って構いません。  
また、Markdownセルを利用し、**なぜ提案手法がうまくいくのか（あるいはうまく行くと考えたのか）を分かりやすく説明し、分類に失敗した記事がある場合は、失敗した理由を議論**して下さい。

なお、精度 (Accuracy) は、未知の記事の総数を$N$、正しく分類できたものの数を$TP$とすると、以下の式で評価するものとします。   
$$\mbox{Accuracy} = \frac{TP}{N}$$   

### ライセンス

本教材で使用するウィキペディアのコンテンツは Creative Commons Attribution-ShareAlike 3.0 Unported License (CC-BY-SA) および GNU Free Documentation License (GFDL) の下にライセンスされています。
本データも同じくこれらのライセンスを継承します。
詳しくは[こちら](https://ja.wikipedia.org/wiki/Wikipedia:%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89)を参照してください。

![CC-BY-SA_icon.svg](https://www.eidos.ic.i.u-tokyo.ac.jp/~sato/assignments/project2/https://upload.wikimedia.org/wikipedia/commons/d/d0/CC-BY-SA_icon.svg)

## Project2. Let's improve the baseline method with your own idea (Advanced exercises)

In the basic exercises, you impremented a code to categorize the six uncategorized articles, which were extracted from the Wikipedia data, using the dataset $D$ as the training data.   
Although your code gives correct category labels to the given articles in the basic exercises, you may notice that it is not always successful but some articles are mis-categorized when you apply the method to various articles.

In this advanced exercises, consider the method implemented in the basic exercises as a baseline method and improve it so that it achieves higher accuracy than the original one.   
**Please report the results of implementing your own ideas and comparing and evaluating the accuracy of the baseline method with your proposed method.**.   

You can use diagrams and tables to illustrate, as appropriate.   
Also, using the Markdown cell, **explain clearly why the proposed method works (or you thought it would work), and if there are articles that failed to be categorized, discuss the reasons for the failure.**

The accuracy is evaluated by the following equation;  
$$\mbox{Accuracy} = \frac{TP}{N},$$   
where $N$ is the total number of the uncategorized articles and $TP$ is the number of articles that are categorized correctly.


### Licenses

The Wikipedia contents used in this learning material are licensed
under Creative Commons Attribution-ShareAlike 3.0 Unported License (CC-BY-SA)
and GNU Free Documentation License (GFDL).
This dataset inherits these licenses as well.
For details, refer to [this site](https://ja.wikipedia.org/wiki/Wikipedia:%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89).

![CC-BY-SA_icon.svg](https://www.eidos.ic.i.u-tokyo.ac.jp/~sato/assignments/project2/https://upload.wikimedia.org/wikipedia/commons/d/d0/CC-BY-SA_icon.svg)

## 準備：ウィキペディアデータセットの読み込み

以下のコードを使って、データセット $D$ を辞書 `Dw` に読み込んでください。
これは基礎課題のものと同じです。

`Dw` のキーがカテゴリ `cate` のとき、`Dw[cate]` は、カテゴリ `cate` のすべての記事の重要語リストを連結して得られるリストを与えます。 ここで、カテゴリとは、冒頭で述べた6つのカテゴリ (`animal`, `art`, `economy`, `law`, `plant`, `politics`) のいずれかです。

## Preparation: Reading the Wikipedia dataset

Execute the following code, and load the dataset $D$ into the dictionary `Dw`.   
This code is excerpted from the basic exercises.

If a key of `Dw` is a category `cate`, `Dw[cate]` gives
the list obtained by concatenating the lists of important words of all the articles in the category `cate`.
Here, a category is one of the six categories:
`animal`, `art`, `economy`, `law`, `plant` and `politics`.

In [2]:
### Execute the following code to load the Wikipedia dataset:

Dw = {}

with open('text/wiki_dataset.csv', 'r', encoding='utf-8') as fi:
    fi.readline()
    for line in fi:
        tmp = line.replace('\\n', '\n').split('\t')
        if tmp[0] not in Dw:
            Dw[tmp[0]] = []
        Dw[tmp[0]].extend(tmp[2].split(' '))

各カテゴリごとに最初の10個の重要語を表示してみましょう。

Let's print the first ten important words of each category.

In [3]:
### Given code:

for cate in Dw:
    print('Category：', cate)
    print('Important words:', Dw[cate][:10])

Category： animal
Important words: ['アドレナリン', 'アドレナリン', 'adrenaline', '英名', 'アドレナリン', '米', '名', 'エピネフリン', 'IUPAC', '組織']
Category： art
Important words: ['固体', '撮像', '素子', '固体', '撮像', '素子', 'こ', 'たいす', 'うそ', 'する']
Category： economy
Important words: ['ミクロ', '経済', '学', 'ミクロ', '経済', '学', 'ミクロ', 'けいざい', 'がく', 'マクロ']
Category： law
Important words: ['首都', '首都', 'する', 'ゅと', '一', '国', '中心', 'なる', '都市', 'こと']
Category： plant
Important words: ['サクラ', 'サクラ', '桜', 'バラ', '科', 'モモ', '亜', '科', 'スモモ', '属']
Category： politics
Important words: ['首都', '首都', 'する', 'ゅと', '一', '国', '中心', 'なる', '都市', 'こと']


## 準備：未知の記事集合およびその正解のカテゴリラベルの読み込み

CSVファイル `text/test_data.csv` には分類対象であるカテゴリが未知の記事60本が納められています。   
ただし、本文はあらかじめデータセット`D`と同様に重要語リストに変換されています。    

以下のコードを用いて、これらの記事を、各記事のタイトルをキー、その本文の重要語リストを値とする辞書 `Aw2` に読み込んでください。   
また同時に、各記事のタイトルをキー、その正解のカテゴリ名を値とする辞書`Aw2_ans`に読み込んでください。

正解率は、推定したカテゴリラベルを正解のカテゴリラベルと比較することで評価します。   
よって当然ながら、この正解のカテゴリラベルを、ラベルの推定のために使ってはいけません。


## Preparation: Loading a set of the uncategorized aticles and their correct category labels

The CSV file `text/test_data.csv` contains 60 target articles whose category is unknown.   
The bodies are converted to a list of important words in the same way as the data set `D` in advance.    

Execute the following code to load these uncategorized articles into the dictionary `Aw2` with the title of each article as a key and the list of important words in its body as the corresponding value.   
At the same time, create the dictionary `Aw2_ans` with the title of each article as a key and the correct category label as the value.

The accuracy is evaluated by comparing the estimated category label with the correct category label.   
Needless to say, the correct category labels must not be used to estimate the category labels.


In [4]:
### Given code:

Aw2 = {}
Aw2_ans = {}

with open('text/test_data.csv', 'r', encoding='utf-8') as fi:
    fi.readline()
    for line in fi:
        tmp = line.replace('\\n', '').split('\t')
        Aw2[tmp[1]] = tmp[2].split(' ')
        Aw2_ans[tmp[1]] = tmp[0]

以下のコードで`Aw2`および`Aw2_ans`を1つ書き出してみましょう。  

The contents of `Aw2` and `Aw2_ans` are printed as follows.

In [5]:
### Given code:

for title in Aw2:
    print('title:', title)
    print('Correct answer label:', Aw2_ans[title])
    print('Important words:', Aw2[title])
    break

title: ツチノコ
Correct answer label: animal
Important words: ['ツチノコ', 'ツチノコ', '槌', '子', '日本', '生息', 'する', '言い伝える', '未', '確認', '動物', '(', 'UMA', ')', 'ひとつ', '鎚', '似る', '形態', '胴', '太い', 'ヘビ', '形容', 'する', '北海道', '南西諸島', '除く', '日本', '全国', '目撃', '例', 'ある', 'する', 'テレビ', '番組', 'ツチノコ', '捕獲', '成功', 'する', 'する', '話', '取り上げる', '祟る', '恐れる', '逃がす', '死体', '消える', '理由', '追求', '打ち切る', '客観', '的', '存在', '証明', 'できる', '事例', '無い', 'ツチノコ', '名称', '京都', '府', '三重', '県', '奈良', '県', '四国', '北部', '用いる', '方言', '東北', '地方', 'バチヘビ', '呼ぶ', 'ほか', 'ノヅチ', 'タテクリカエシ', 'ツチンボ', 'ツチ', 'ヘビ', '土', '転び', '日本', '全国', '40', '種', '呼称', 'ある', 'ノヅチ', '土', '転び', '別', '妖怪', '独立', 'する', '例', 'ある', 'いずれ', 'ツチノコ', '生け捕り', '条件', 'なる', '現在', '賞金', '継続', 'する', '不問', 'する\n']


## 皆さんのコードおよび解説

以下で皆さんのコードやその解説、結果の評価および議論を行ってください。   
- この'project2.ipynb'は自動採点されません．答案検査システムもありません。教員やTAが一つずつ見て採点します。
- 解説や議論はMarkdownセルに、コードはCodeセルに記入してください。
- 提出されたipynbファイルは教員のPCで実行したうえで評価します。実行に必要な追加パッケージがあれば指定するなどして、実行できるファイルを提出してください。
- Codeセル、Markdownセルは必要に応じて増やして構いません

## Describe your code and explain it
Describe your code, explanation and discussion below.
- This notebook 'project2.ipynb' will not be automatically graded at all. No automatic checking for it is provided. The faculty members and TAs will read and execute this notebook and give a grade manually.
- Fill the explanation and discussion of your method in Markdown cells. The code should be written in Code cells.
- The submitted notebook will be executed on the faculty member's PC before grading. Please submit an executable file, specifying all additional packages required for execution if any.
- You can add Code cells and Markdown cells as needed.

# 既存手法の精度検証

提案手法の性能評価のため、既存手法 (基礎課題で実装した手法)を用いて、 `./text/test_data.csv` をカテゴリ分類する。

In [6]:
def compute_word_frequency(dw):
    import itertools
    from collections import Counter
    important_word_list = list(itertools.chain.from_iterable(dw.values()))
    return dict(Counter(important_word_list))

W = compute_word_frequency(Dw)

In [7]:
def extract_frequent_words(word_frequency, coverage):
    n_freq = 0
    answer = []
    for word, freq in sorted(word_frequency.items(), key=lambda x: -x[1]):
        if 1.0 * n_freq / sum(word_frequency.values()) < coverage:
            n_freq += freq
            answer.append(word)
        else:
            break
    return answer

F = extract_frequent_words(W, 0.5)

In [9]:
def words2vec(words, frequent_words):
    counter = dict([[fw, 0] for fw in frequent_words])
    for word in words:
        if counter.get(word) is not None:
            counter[word] += 1
    return [counter[fw] for fw in frequent_words]

Av2 = {}
for title, words in Aw2.items():
    Av2[title] = words2vec(words, F)

Dv = {}
for cate, words in Dw.items():
    Dv[cate] = words2vec(words, F)

In [12]:
import numpy as np

def guess_category(dv, v):
    def cos_sim(x, y):
        return np.sum(x * y) / (np.sqrt(np.sum(x * x)) * np.sqrt(np.sum(y * y)))
    cos_sim_dict = dict([[cate, cos_sim(np.array(x), np.array(v))]
                         for cate, x in dv.items()])
    return max(cos_sim_dict, key=cos_sim_dict.get)

results = []
for title in Av2.keys():
    pred = guess_category(Dv, Av2[title])
    results.append(pred == Aw2_ans[title])
    
accuracy = 1.0 * sum(results) / len(results)
print("Accuracy: {:.2%}".format(accuracy))
print()

Accuracy: 81.67%


# 提案手法の概要 / Outline of your proposed method

...


# 着想に至った経緯 / Background to the idea

...

# 処理の流れ / Processing flow

1. First step...
1. Second step...
1. Third step...


In [ ]:
# 提案手法のコード / The code of your proposed method
# 注意: 適宜、コメント行として解説を書き込み、わかりやすいコードとなるように務めてください。
# Note: Write commentaries as comment lines where appropriate and try to make the code easy to understand.
...

# 評価 / Evaluation

... 

In [ ]:
# 提案手法の評価に関するコード / The code for evaluation of your method
...

# 議論と結論 / Discussion and conclusion

...